In [1]:
] activate "/Users/anarres/Documents/projects/BioXP"

 Activating environment at `~/Documents/projects/BioXP/Project.toml`


Some visual sugar

In [2]:
# ] add ProgressMeter

In [3]:
using BioXP
using ProgressMeter
using Random
using JSON
# using BenchmarkTools

### Inputs

In [4]:
## Organism JSONs from ecg (JGI)
input_dir = "data/input/rids-methanogens/"

## Master file from ecg (with dgs added from `add_dgs_to_master` .py files)
rstructs_path = "data/input/rstructs/master_from_redges-og-submission.json"

## User defined seeds/targets
seeds_and_targets_path = "data/input/seeds/seeds.json"
sid_name = "Enceladus_20-SAFR-032"
tid_name = "targets_Freilich09";

#### Seed set randomization

In [5]:
compound_structs_path = "data/input/compound/"
n_runs = 100
n_swaps = 1000
beta = 20
sortkey = :exact_mass
zero_mass_behavior = "end"
rng_seed = 1234
# rng = MersenneTwister(1234);

### Path to write to

In [6]:
write_dir = "data/input/sid_randomizations-methanogens/";

if !ispath(write_dir)
    mkpath(write_dir)
end

"data/input/sid_randomizations-methanogens"

### Make random seeds

In [30]:
rstructs = readmaster(rstructs_path)
compound_structs = readcompounds(compound_structs_path);

for path in readdir(input_dir)
    
    org_dir = joinpath(input_dir,path)
    
    if isdir(org_dir)
        
        if !ispath(joinpath(write_dir,path))
            mkpath(joinpath(write_dir,path))
        end
        
        p = Progress(length(readdir(org_dir)),desc="$path")
        
        Threads.@threads for (i,fname) in collect(enumerate(readdir(org_dir)))
            if endswith(fname,".json")

                rids = readids(joinpath(org_dir,fname))
#                 cids = list_biosystem_compounds_from_rids(rstructs,rids)
                cids = list_biosystem_compounds_from_rids(rstructs,BioXP.remove_rids_not_in_rstructs(rstructs,rids))
#                 cids = BioXP.identify_biosystem_compounds(rstructs,rids)
                write_path = joinpath(write_dir,basename(org_dir),fname)

                random_seed_sets = randomizecompounds(cids,
                                        compound_structs,
                                        n_runs,
                                        n_swaps,
                                        beta,
                                        sortkey,
                                        zero_mass_behavior,
                                        MersenneTwister(rng_seed))
                open(write_path,"w") do f
                    JSON.print(f, random_seed_sets, 2) #indent=2
                end

                next!(p)
            end
        end
    end
end


archaea 86%|█████████████████████████████████████▊      |  ETA: 0:00:00